## 한글 문서의 분류
네이버 뉴스 데이터를 크롤링하여 분류 연습<br>
기사 내용과 기사를 쓴 뉴스 회사의 이름을 학습하여 기사가 어느 뉴스회사의 기사인지 예측하고자 함
### data file 내용
'국민일보', '전자신문', '한겨레', TV조선' 등 네이버 뉴스 크롤링을 하여 총 3865개의 기사를 수집
csv 파일 안에 뉴스 회사 이름, 기사내용 의 순으로 저장되어 있음

In [2]:
import csv

text = []
y = []
with open('navernews_utf2.csv', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        #print(row)
        if row: #그 줄에 내용이 있는 경우에만
            text.append(row[1]) #기사를 text 리스트에 추가
            y.append(row[0]) #뉴스 회사이름을 text 리스트에 추가

In [28]:
print('Num of samples: {}'.format(len(text)))
print('뉴스 이름: {}'.format(set(y)))

Num of samples: 3865
뉴스 이름: {'국민일보', '전자신문', '한겨레', 'TV조선', '서울경제', '파이낸셜뉴스', '머니투데이', '매일신문', '한국경제', '조선일보', '아이뉴스24', '노컷뉴스', '세계일보', '동아일보', 'press', '이데일리', '데일리안', '아시아경제', '스포츠서울', '경향신문', '뉴스1', '디지털타임스', '뉴시스', '헤럴드경제', '한국일보', '연합뉴스'}


In [4]:
from sklearn.model_selection import train_test_split

# split data and labels into a training and a test set
X_train, X_test, y_train, y_test = train_test_split(text, y, random_state=0)
# 비율을 지정하지 않으면 75:25로 분할됨

In [5]:
len(X_train) #3865의 0.75

2898

In [6]:
from konlpy.tag import Okt #konlpy에서 Twitter 형태소 분석기를 import
#from konlpy.tag import Twitter #konlpy에서 Twitter 형태소 분석기를 import
twitter_tag = Okt()
#twitter_tag = Twitter()

In [7]:
print(twitter_tag.morphs(X_train[1])) #둘째 기사에 대해 형태소 단위로 tokenize

['전국', '장애인', '부모', '연대', '회원', '들', '이', '2일', '오전', '발달장애', '주간', '활동', '서비스', '도입', '을', '촉구', '하며', '청와대', '춘추관', '진입', '을', '시도', '하다', '실패한', '후', '문재인', '대통령', '면담', '요청서', '를', '청와대', '관계자', '에게', '전달', '하고', '있다', '.', '청와대', '사진기', '자단', '▶', '네이버', '메인', '에서', '받아', '보기', '▶', '두고', '두고', '읽는', '뉴스', '▶', '인기', '무료', '만화', '©', '(www.khan.co.kr', '),', '무단', '전', '재', '및', '재', '배포', '금지']


In [8]:
twitter_tag.nouns(X_train[1]) #둘째 기사에서 명사만 추출

['전국',
 '장애인',
 '부모',
 '연대',
 '회원',
 '오전',
 '발달장애',
 '주간',
 '활동',
 '서비스',
 '도입',
 '촉구',
 '청와대',
 '춘추관',
 '진입',
 '시도',
 '후',
 '문재인',
 '대통령',
 '면담',
 '요청서',
 '청와대',
 '관계자',
 '전달',
 '청와대',
 '사진기',
 '자단',
 '네이버',
 '메인',
 '보기',
 '뉴스',
 '인기',
 '무료',
 '만화',
 '무단',
 '재',
 '및',
 '재',
 '배포',
 '금지']

In [9]:
def twit_tokenizer(text): # Twitter 형태소 분석기의 명사추출함수를 tokenizer 함수로 사용
    return twitter_tag.nouns(text)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=2) #Twitter 형태소분석기에서 명사만 추출하는 함수를 tokenizer로 이용
# twit_tokenizer 대신 twitter_tag.nouns를 직접 써도 됨
# 하나의 문서에서만 출현한 단어는 쓸모가 없으므로 제외, 즉 최소 document frequency를 2로 설정

X_train_tfidf = tfidf.fit_transform(X_train) # train data 변환 -> tfidf vector
X_test_tfidf = tfidf.transform(X_test) # test data 변환 -> tfidf vector

clf = LogisticRegression() # logistic regression 분류기 선언
clf.fit(X_train_tfidf, y_train) # 분류기 학습
print('Train score', clf.score(X_train_tfidf, y_train)) # train data 예측정확도
print('Test score', clf.score(X_test_tfidf, y_test)) # test data 예측정확도
print(X_train_tfidf.shape) # 총 8555개의 명사로 이루어짐

Train score 0.943064182194617
Test score 0.8748707342295761
(2898, 8555)


In [11]:
X_test[:10] #test data에서 앞 10개를 출력

['(서울=) 문무일 검찰총장이 검·경 수사권 조정 법안 등의 패스트트랙(신속처리안건) 지정을 비판해 파문을 일으켰다. 해외 순방 중인 문 총장은 지난 1일 대검을 통해 낸 입장자료에서 "신속처리안건으로 지정된 법안들은 견제와 균형이라는 민주주의 원리에 반한다"며 "검찰총장으로서 우려를 금할 수 없다"고 밝혔다. 문 총장이 권력기관 개혁 법안의 패스트트랙 지정에 검찰 수장으로서 우려의 목소리를 낸 것은 검찰 조직에서는 환영받을지 모르겠다. 그러나 국민의 눈에는 대의기관인 국회의 결정에 뒤늦게 제동을 하는 듯한 모습으로 비쳐 부적절해 보인다. 문 총장이 임기를 두달여 남긴 시점에 반기를 드는 듯한 행보를 보이는 것은 이해하기 어렵다. 정의당 이정미 대표는 2일 문 총장을 겨냥해 "국회의 정당한 입법절차에 정부 관료가 공공연히 반기 드는 것이야말로 민주주의 원리를 망각한 행동"이라며 "문 총장의 언행은 기득권을 포기하지 못하는 검찰 권력의 현실을 그대로 보여준다"고 직격탄을 날렸다.권력기관 개혁은 검찰이 자초한 측면이 크다. 형사사건 수사, 수사지휘, 기소권을 모두 갖는 소추(訴追) 기관인 검찰은 그동안 국민 편에 서기보다는 권력 편에서 막강한 권한을 휘두르며 국민 위에 군림해왔다. 김학의 전 법무부 차관의 \'별장 성폭행\' 사건이 불거졌을 때 납득하기 어려운 사유로 끝내 무혐의 처분한 조직이 바로 검찰이다. \'권력의 시녀\'라는 말이 그냥 나온 게 아니다. 검찰은 본연의 업무라 할 수 있는 경찰 형사사건 수사지휘보다 공안, 특수, 강력, 금융 등으로 직접수사 범위를 확대해가며 기득권 유지와 강화에 매달려왔다. 국민의 검찰개혁 요구는 검찰의 자업자득인 셈이다.그렇지만 검·경 수사권 조정법과 공수처 설치법 등 권력기관 개혁법안은 보완할 필요는 있다는 전문가 견해가 적지 않다. 여당인 더불어민주당의 조응천 의원은 패스트트랙으로 지정된 수사권조정법안이 그대로 통과될 경우 국내 정보업무를 전담하는 경찰이 통제를 거의 안 받고 1차 수사권을 행사함으로써 \'경찰국가화\

In [12]:
clf.predict(X_test_tfidf[:10]) # test data의 앞 10개에 대한 예측내용

array(['연합뉴스', '노컷뉴스', '연합뉴스', '뉴시스', '머니투데이', '연합뉴스', '뉴스1', '이데일리',
       '뉴시스', '연합뉴스'], dtype='<U6')

In [13]:
print(y_test[:10]) # test data 앞 10개의 실제 뉴스회사 이름

['연합뉴스', '노컷뉴스', '연합뉴스', '뉴시스', '머니투데이', '연합뉴스', '뉴스1', '이데일리', '머니투데이', '뉴스1']


### 성능을 개선하기 위한 노력

In [14]:
# morphs()는 명사 외에도 모든 형태소를 포함
print(twitter_tag.morphs(X_train[1]))

['전국', '장애인', '부모', '연대', '회원', '들', '이', '2일', '오전', '발달장애', '주간', '활동', '서비스', '도입', '을', '촉구', '하며', '청와대', '춘추관', '진입', '을', '시도', '하다', '실패한', '후', '문재인', '대통령', '면담', '요청서', '를', '청와대', '관계자', '에게', '전달', '하고', '있다', '.', '청와대', '사진기', '자단', '▶', '네이버', '메인', '에서', '받아', '보기', '▶', '두고', '두고', '읽는', '뉴스', '▶', '인기', '무료', '만화', '©', '(www.khan.co.kr', '),', '무단', '전', '재', '및', '재', '배포', '금지']


In [15]:
tfidf = TfidfVectorizer(tokenizer=twitter_tag.morphs, min_df=2) # 명사 대신 모든 형태소를 사용
#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)
#명사만 사용한 것에 비해 train score는 상승, test score는 하락

Train score 0.9582470669427191
Test score 0.9327817993795243
(2898, 23380)


In [16]:
print(twitter_tag.pos(X_train[1], norm=True, stem=True)) #pos()는 형태소와 품사를 함께 제공

[('전국', 'Noun'), ('장애인', 'Noun'), ('부모', 'Noun'), ('연대', 'Noun'), ('회원', 'Noun'), ('들', 'Suffix'), ('이', 'Josa'), ('2일', 'Number'), ('오전', 'Noun'), ('발달장애', 'Noun'), ('주간', 'Noun'), ('활동', 'Noun'), ('서비스', 'Noun'), ('도입', 'Noun'), ('을', 'Josa'), ('촉구', 'Noun'), ('하다', 'Verb'), ('청와대', 'Noun'), ('춘추관', 'Noun'), ('진입', 'Noun'), ('을', 'Josa'), ('시도', 'Noun'), ('하다', 'Verb'), ('실패하다', 'Adjective'), ('후', 'Noun'), ('문재인', 'Noun'), ('대통령', 'Noun'), ('면담', 'Noun'), ('요청서', 'Noun'), ('를', 'Josa'), ('청와대', 'Noun'), ('관계자', 'Noun'), ('에게', 'Josa'), ('전달', 'Noun'), ('하다', 'Verb'), ('있다', 'Adjective'), ('.', 'Punctuation'), ('청와대', 'Noun'), ('사진기', 'Noun'), ('자단', 'Noun'), ('▶', 'Foreign'), ('네이버', 'Noun'), ('메인', 'Noun'), ('에서', 'Josa'), ('받다', 'Verb'), ('보기', 'Noun'), ('▶', 'Foreign'), ('두다', 'Verb'), ('두다', 'Verb'), ('읽다', 'Verb'), ('뉴스', 'Noun'), ('▶', 'Foreign'), ('인기', 'Noun'), ('무료', 'Noun'), ('만화', 'Noun'), ('©', 'Foreign'), ('(www.khan.co.kr', 'URL'), ('),', 'Punctuation'), ('무단', 'Noun')

In [17]:
def twit_tokenizer2(text): #전체를 다 사용하는 대신, 명사, 동사, 형용사를 사용
    target_tags = ['Noun', 'Verb', 'Adjective']
    result = []
    for word, tag in twitter_tag.pos(text, norm=True, stem=True):
        if tag in target_tags:
            result.append(word)
#            result.append('/'.join([word, tag]))
    return result

In [18]:
print(twit_tokenizer2(X_train[1])) # 사용 예

['전국', '장애인', '부모', '연대', '회원', '오전', '발달장애', '주간', '활동', '서비스', '도입', '촉구', '하다', '청와대', '춘추관', '진입', '시도', '하다', '실패하다', '후', '문재인', '대통령', '면담', '요청서', '청와대', '관계자', '전달', '하다', '있다', '청와대', '사진기', '자단', '네이버', '메인', '받다', '보기', '두다', '두다', '읽다', '뉴스', '인기', '무료', '만화', '무단', '재', '및', '재', '배포', '금지']


In [19]:
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer2, min_df=2) #명사, 동사, 형용사를 이용하여 tfidf 생성
#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)
# 현재까지 중에서 test score가 가장 뛰어남

Train score 0.9406487232574189
Test score 0.8738366080661841
(2898, 9687)


In [20]:
# 모든 형태소를 다 사용하고 품사를 알 수 있도록 하면?
def twit_tokenizer3(text):
    #target_tags = ['Noun', 'Verb', 'Adjective']
    result = []
    for word, tag in twitter_tag.pos(text, norm=True, stem=True):
        result.append('/'.join([word, tag])) #단어의 품사를 구분할 수 있도록 함
    return result

In [21]:
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer3, min_df=2)
#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)
#성능이 오히려 떨어지고 품사 표시 없이 전체를 다 사용한 경우에 비해 train은 떨어지고, test는 올라감

Train score 0.9565217391304348
Test score 0.9234746639089969
(2898, 19268)


In [22]:
# train score가 높으므로 ridge를 쓰면 어떨까?
from sklearn.linear_model import RidgeClassifier
ridge_clf = RidgeClassifier(alpha = 1)
ridge_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))
# train score가 올라가는 현상이 벌어짐
# test score가 올라갔으나 명사, 형용사, 동사를 쓴 것보다 떨어짐

Train set score: 0.995
Test set score: 0.988


In [23]:
#lasso를 쓰면?
from sklearn.linear_model import LogisticRegression
import numpy as np
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear')
lasso_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))
print('Used features count: {}'.format(np.sum(lasso_clf.coef_ != 0)), 'out of', X_train_tfidf.shape[1])

Train set score: 0.972
Test set score: 0.960
Used features count: 151 out of 19268


In [29]:
#lsa를 쓰면?
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=150, n_iter=7, random_state=42) #압축할 component의 수 지정
svd.fit(X_train_tfidf)  
print(svd.explained_variance_ratio_)  #계산된 각 component가 설명하는 분산의 비율
print(svd.explained_variance_ratio_.sum())  #선택된 component들이 설명하는 분산의 합 -> 선택한 component의 수에 따라 달라짐
print(svd.singular_values_) 
print(svd.components_.shape)

[0.01749844 0.06259259 0.04867669 0.02820975 0.0261339  0.02292455
 0.0216244  0.02032561 0.01799395 0.01646622 0.01590068 0.01329175
 0.01186406 0.01054471 0.01043772 0.00999851 0.00980958 0.00949685
 0.00899328 0.00839679 0.00814633 0.00797232 0.00764513 0.007358
 0.00710842 0.00696733 0.00664081 0.00632563 0.00624871 0.00590439
 0.00587019 0.00545757 0.00539159 0.00530238 0.00501105 0.00480495
 0.00471938 0.00453381 0.004455   0.00435139 0.00422777 0.0040918
 0.00396747 0.0039218  0.00377193 0.00373484 0.00368934 0.00358876
 0.00354618 0.00346198 0.00345037 0.00334575 0.00331011 0.00322603
 0.00320981 0.00315917 0.00310058 0.00306519 0.00301535 0.00295869
 0.00294814 0.0028852  0.00286418 0.00278192 0.00273721 0.00268792
 0.0026802  0.00262345 0.00255143 0.00251287 0.00248258 0.00245528
 0.00241811 0.00236538 0.00234642 0.0023207  0.00225045 0.00224059
 0.00218883 0.00217195 0.00213246 0.00211904 0.00208108 0.0020428
 0.0020308  0.00199983 0.00199294 0.00194749 0.00193893 0.00189195

In [25]:
X_train_svd = svd.transform(X_train_tfidf) #선택된 component를 이용하여 19000개의 feature로부터 feature extract (dimension reduce)
X_test_svd = svd.transform(X_test_tfidf)

from sklearn.linear_model import LogisticRegression
SVD_clf = LogisticRegression()
SVD_clf.fit(X_train_svd, y_train)
print('Train set score: {:.3f}'.format(SVD_clf.score(X_train_svd, y_train)))
print('Test set score: {:.3f}'.format(SVD_clf.score(X_test_svd, y_test)))

Train set score: 0.929
Test set score: 0.904


## 높은 정확도의 이유.

정확도가 높게 나온 이유는 기사 내용에 뉴스 사이트, 기자의 이름 등의 패턴으로 학습이 되었기 때문이라 생각됨.
기사에 있는 뉴스 회사 이름은 전처리 과정에서 삭제하였으나, '기자명@chosun' 등 기자의 이메일이나 www.경향신문.com 과 같은 주소명이 기사내용에 남아 있었음.
추후 전처리를 통해 제대로된 분류를 해볼 계획임.